In [6]:
import numpy as np
import os
import scipy.io
from sklearn.metrics import classification_report,confusion_matrix

In [7]:
#Please add the folder name of the dataset to run it on different dataset.
dataset = 'CUB'

In [8]:
#Replace Path to run on your own system
res101 = scipy.io.loadmat('C:/Users/Zach/Documents/NorthEastern/SmallDataMachineLearning/Code/ESZSL/Data/xlsa17/data/'+dataset+'/res101.mat')
att_splits = scipy.io.loadmat('C:/Users/Zach/Documents/NorthEastern/SmallDataMachineLearning/Code/ESZSL/Data/xlsa17/data/'+dataset+'/att_splits.mat')

In [9]:
#Using the correct naming conventions to get the loctions
trainval_loc = 'trainval_loc'
train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'

In [10]:
#RES101 is Resnet 101, this is the output from CNN
labels = res101['labels']
#Array of training labels (class label)
labels_train = labels[np.squeeze(att_splits[train_loc]-1)]
#array of validation labels
labels_val = labels[np.squeeze(att_splits[val_loc]-1)]
#train + validation labels
labels_trainval = labels[np.squeeze(att_splits[trainval_loc]-1)]
#test labels
labels_test = labels[np.squeeze(att_splits[test_loc]-1)]

In [11]:
#list all unique labels, should be 200 for birds
unique_labels = np.unique(labels)

In [12]:
train_labels_seen = np.unique(labels_train)
val_labels_unseen = np.unique(labels_val)
trainval_labels_seen = np.unique(labels_trainval)
test_labels_unseen = np.unique(labels_test)

In [13]:
#label_train now should be an index into the train_labels seen
#example: anything that had 200 as a label, now is 99 which is the last index in train_labels seen
#which aligns with label 200
i = 0
for labels in train_labels_seen:
    labels_train[labels_train == labels] = i    
    i = i+1
j = 0
for labels in val_labels_unseen:
    labels_val[labels_val == labels] = j
    j = j+1
k = 0
for labels in trainval_labels_seen:
    labels_trainval[labels_trainval == labels] = k
    k = k+1
l = 0
for labels in test_labels_unseen:
    labels_test[labels_test == labels] = l
    l = l+1

In [14]:
#Get all features of all data
X_features = res101['features']
#select those for the training
train_vec = X_features[:,np.squeeze(att_splits[train_loc]-1)]
#validation set
val_vec = X_features[:,np.squeeze(att_splits[val_loc]-1)]
#train+validation
trainval_vec = X_features[:,np.squeeze(att_splits[trainval_loc]-1)]
#last the test features
test_vec = X_features[:,np.squeeze(att_splits[test_loc]-1)]

In [15]:
#Attribute Signature matrix (all)
#All normalized to have unit 12 norm
signature = att_splits['att']
#getting the training,validation,train+val,and the test signatures
train_sig = signature[:,(train_labels_seen)-1]
val_sig = signature[:,(val_labels_unseen)-1]
trainval_sig = signature[:,(trainval_labels_seen)-1]
test_sig = signature[:,(test_labels_unseen)-1]

In [16]:
#params for train and val set
m_train = labels_train.shape[0]
n_val = labels_val.shape[0]
z_train = len(train_labels_seen)
z1_val = len(val_labels_unseen)

#params for trainval and test set
m_trainval = labels_trainval.shape[0]
n_test = labels_test.shape[0]
z_trainval = len(trainval_labels_seen)
z1_test = len(test_labels_unseen)

In [17]:
#Setting ground truths for the the classes
#ground truth for train 
gt_train = 0*np.ones((m_train, z_train))
gt_train[np.arange(m_train), np.squeeze(labels_train)] = 1

#grountruth for trainval 
gt_trainval = 0*np.ones((m_trainval, z_trainval))
gt_trainval[np.arange(m_trainval), np.squeeze(labels_trainval)] = 1

#train set dimensions
d_train = train_vec.shape[0] #num of dimensions of features
a_train = train_sig.shape[0] #num of attributes

In [18]:
'''
I am trying to create a Neural Network that will take the features from the images
and predict the attributes.
'''

import torch
import torch.nn as nn
from sklearn.preprocessing import normalize 


#going to reassign
X = torch.tensor(train_vec)
S = torch.tensor(train_sig)
Y = torch.tensor(gt_train)

# number of features from image
inputDim  = X.shape[0]
#Number of classes
outputDim = S.shape[0]

S.cuda()


tensor([[0.0106, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0438],
        [0.0106, 0.0113, 0.0351,  ..., 0.0033, 0.1118, 0.0281],
        [0.0071, 0.0094, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0092, 0.0027, 0.0028,  ..., 0.0056, 0.0821, 0.0602],
        [0.0253, 0.0213, 0.0151,  ..., 0.0000, 0.0584, 0.0770],
        [0.0207, 0.0586, 0.1582,  ..., 0.1503, 0.0182, 0.0619]],
       device='cuda:0', dtype=torch.float64)

In [19]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")



Using cuda device
